In [402]:
import tensorflow as tf
import cv2
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
from tensorflow.keras.models import load_model

In [403]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)

In [404]:
data = tf.keras.utils.image_dataset_from_directory('data')

Found 193 files belonging to 2 classes.


In [405]:
data_iterator = data.as_numpy_iterator()

In [406]:
batch = data_iterator.next()

In [407]:
data = data.map(lambda x,y: (x/255, y))

In [408]:
train_size = int(len(data)*.7)
validation_size = int(len(data)*.2)
test_size = int(len(data)*.1)

In [410]:
train = data.take(train_size)
validationData = data.skip(train_size).take(validation_size)
test = data.skip(train_size+validation_size).take(test_size)

In [411]:
model = Sequential()

In [412]:
model.add(Conv2D(16, (3,3), 1, activation='relu', input_shape=(256,256,3)))
model.add(MaxPooling2D())
model.add(Conv2D(32, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(16, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [413]:
model.compile('adam', loss=tf.losses.BinaryCrossentropy(), metrics=['accuracy'])

In [414]:
logs='logs'

In [415]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logs)

In [416]:
hist = model.fit(train, epochs=50, validation_data=validationData, callbacks=[tensorboard_callback])

Epoch 1/50
4/4 [==============================] - 5s 944ms/step - loss: 2.1313 - accuracy: 0.4844 - val_loss: 1.5488 - val_accuracy: 0.5000
Epoch 2/50
4/4 [==============================] - 4s 873ms/step - loss: 0.9557 - accuracy: 0.5625 - val_loss: 0.6552 - val_accuracy: 0.5625
Epoch 3/50
4/4 [==============================] - 4s 871ms/step - loss: 0.6741 - accuracy: 0.6797 - val_loss: 0.6463 - val_accuracy: 0.6875
Epoch 4/50
4/4 [==============================] - 4s 858ms/step - loss: 0.6263 - accuracy: 0.5703 - val_loss: 0.6022 - val_accuracy: 0.5625
Epoch 5/50
4/4 [==============================] - 4s 869ms/step - loss: 0.5264 - accuracy: 0.6094 - val_loss: 0.5535 - val_accuracy: 0.5938
Epoch 6/50
4/4 [==============================] - 4s 876ms/step - loss: 0.4982 - accuracy: 0.7500 - val_loss: 0.5516 - val_accuracy: 0.8438
Epoch 7/50
4/4 [==============================] - 4s 876ms/step - loss: 0.4480 - accuracy: 0.8672 - val_loss: 0.3210 - val_accuracy: 0.9062
Epoch 8/50
4/4 [====

In [417]:
model.save(os.path.join('model','planeOrCarModel.h5'))

In [418]:
new_planeOrCarModel = load_model(os.path.join('model','planeOrCarModel.h5'))

# Model Testing

In [451]:
testImage = cv2.imread('zukunftsauto.jpeg')

In [452]:
resizedTestImage = tf.image.resize(testImage, (256,256))

In [453]:
predictionValue = new_planeOrCarModel.predict(np.expand_dims(resizedTestImage/255, 0))

1/1 [==============================] - 0s 30ms/step


In [454]:
if predictionValue > 0.5: 
    print(f'Predicted class is Plane')
elif predictionValue < 0.5:
    print(f'Predicted class is car')

Predicted class is car
